In [348]:
!pip install rdkit

In [349]:
conda install -c conda-forge imbalanced-learn


Note: you may need to restart the kernel to use updated packages.

Solving environment: ...working... done

# All requested packages already installed.



In [125]:
pip install xgboost==0.90

In [ ]:
!pip install flaml

In [2]:
import numpy as np 
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.EState import EStateIndices
from rdkit.Chem import rdMolDescriptors
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from rdkit.Chem import AllChem
from rdkit import DataStructs
from flaml import AutoML

from sklearn import metrics
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Loading and reading required files
train_data = pd.read_csv('train_II.csv')
test_data = pd.read_csv('test_II.csv')
#feature_data = pd.read_csv('/kaggle/input/the-toxicity-prediction-challenge/feamat.csv')

In [4]:
train_data.head()

,Id,Expected
0,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1644,2
1,CCCCCCCCC(=O)C;2451,2
2,CCCCCCCCCC[N+](C)(C)CCCCCCCCCC.[Cl-];1384,2
3,C1CN(C(=N1)N[N+](=O)[O-])CC2=CN=C(C=C2)Cl;16,2
4,[Na+].[I-];1856,2


In [5]:
test_data = test_data.rename(columns = {"x": "Id"})
test_data2=test_data
test_data2.head()

,Id
0,CC1=CC(=C(C=C1)C(C)(C)C)O;1682
1,CCS(=O)(=O)C1=C(N=CC=C1)S(=O)(=O)NC(=O)NC2=NC(...
2,CC1=NC2=CC=CC=C2N1C3CC4CCC(C3)N4CCC5(CCN(CC5)C...
3,CC1=CC(=O)[N-]S(=O)(=O)O1.[K+];1850
4,CCC(C)(C)C(=O)O[C@H]1C[C@H](C=C2[C@H]1[C@H]([C...


In [6]:
#Splitting Id into 2 separate columns
train_data[["Chemical_Id", "Assay_Id"]] = train_data.Id.str.split(";", expand = True)
test_data[["Chemical_Id", "Assay_Id"]] = test_data.Id.str.split(";", expand = True)
train_data.head()

,Id,Expected,Chemical_Id,Assay_Id
0,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1644,2,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O,1644
1,CCCCCCCCC(=O)C;2451,2,CCCCCCCCC(=O)C,2451
2,CCCCCCCCCC[N+](C)(C)CCCCCCCCCC.[Cl-];1384,2,CCCCCCCCCC[N+](C)(C)CCCCCCCCCC.[Cl-],1384
3,C1CN(C(=N1)N[N+](=O)[O-])CC2=CN=C(C=C2)Cl;16,2,C1CN(C(=N1)N[N+](=O)[O-])CC2=CN=C(C=C2)Cl,16
4,[Na+].[I-];1856,2,[Na+].[I-],1856


In [7]:
train_data= train_data[train_data.Chemical_Id != 'F[Si-2](F)(F)(F)(F)F.[Na+].[Na+]']
test_data= test_data[test_data.Chemical_Id != 'F[Si-2](F)(F)(F)(F)F.[Na+].[Na+]']

In [8]:
descriptors=[]
for index,rows in train_data.iterrows():
        mol=rows['Chemical_Id']
        array_id=rows['Assay_Id']
        expected=rows['Expected']
        molecule = Chem.MolFromSmiles(mol)
        #fp = AllChem.GetHashedMorganFingerprint(molecule, 2)
        fp = AllChem.GetMorganFingerprintAsBitVect(molecule, 2, nBits=512)
        #array = np.zeros((1, ), dtype=np.int8)
        #DataStructs.ConvertToNumpyArray(fp, array)
        #fp = AllChem.GetMorganFingerprint(molecule,2)
        mw=Descriptors.MolWt(molecule)
        logp= Descriptors.MolLogP(molecule)
        TPSA= Descriptors.TPSA(molecule)
        hbd = Descriptors.NumHDonors(molecule)
        hba = Descriptors.NumHAcceptors(molecule)
        n_rot_bonds = Descriptors.NumRotatableBonds(molecule)
        n_aromatic_rings = Descriptors.NumAromaticRings(molecule)
        n_rings = Descriptors.RingCount(molecule)
        n_heavy_atoms = Descriptors.HeavyAtomCount(molecule)
        frac_sp3 = Descriptors.FractionCSP3(molecule)
        Slogp3=  Descriptors.SlogP_VSA3(molecule)
        Slogp4=  Descriptors.SlogP_VSA4(molecule)
        Slogp5=  Descriptors.SlogP_VSA5(molecule)
        Slogp6=  Descriptors.SlogP_VSA6(molecule)
        Slogp7=  Descriptors.SlogP_VSA7(molecule)
        Smr3 =Descriptors.SMR_VSA3(molecule)
        Smr4 =Descriptors.SMR_VSA4(molecule)
        Smr5 =Descriptors.SMR_VSA5(molecule)
        Smr6 =Descriptors.SMR_VSA6(molecule)



        #narombond = rdMolDescriptors.CalcNumBridgeheadAtoms(molecule)
        #num_aromatic_atoms = len(molecule.GetAromaticAtoms())
        descriptors.append([array_id,fp,expected,mw,logp,TPSA,hbd,hba,n_rot_bonds,n_aromatic_rings,n_rings,n_heavy_atoms,frac_sp3,Slogp3,Slogp4,Slogp5,Slogp6,Slogp7,Smr3,Smr4,Smr5,Smr6])
            




            
            

    

In [9]:
descriptors1=[]
fp2=[]
for index,rows in test_data.iterrows():
        mol=rows['Chemical_Id']
        array_id=rows['Assay_Id']
        molecule = Chem.MolFromSmiles(mol)
        #fp = AllChem.GetHashedMorganFingerprint(molecule, 2)
        fp = AllChem.GetMorganFingerprintAsBitVect(molecule, 2, nBits=512)
        #array = np.zeros((1, ), dtype=np.int8)
        #DataStructs.ConvertToNumpyArray(fp, array)
        mw=Descriptors.MolWt(molecule)
        logp= Descriptors.MolLogP(molecule)
        TPSA= Descriptors.TPSA(molecule)
        hbd = Descriptors.NumHDonors(molecule)
        hba = Descriptors.NumHAcceptors(molecule)
        n_rot_bonds = Descriptors.NumRotatableBonds(molecule)
        n_aromatic_rings = Descriptors.NumAromaticRings(molecule)
        n_rings = Descriptors.RingCount(molecule)
        n_heavy_atoms = Descriptors.HeavyAtomCount(molecule)
        frac_sp3 = Descriptors.FractionCSP3(molecule)
        Slogp3=  Descriptors.SlogP_VSA3(molecule)
        Slogp4=  Descriptors.SlogP_VSA4(molecule)
        Slogp5=  Descriptors.SlogP_VSA5(molecule)
        Slogp6=  Descriptors.SlogP_VSA6(molecule)
        Slogp7=  Descriptors.SlogP_VSA7(molecule)
        Smr3 =Descriptors.SMR_VSA3(molecule)
        Smr4 =Descriptors.SMR_VSA4(molecule)
        Smr5 =Descriptors.SMR_VSA5(molecule)
        Smr6 =Descriptors.SMR_VSA6(molecule)
        #narombond = rdMolDescriptors.CalcNumBridgeheadAtoms(molecule)
        #num_aromatic_atoms = len(molecule.GetAromaticAtoms())
        descriptors1.append([array_id,fp,mw,logp,TPSA,hbd,hba,n_rot_bonds,n_aromatic_rings,n_rings,n_heavy_atoms,frac_sp3,Slogp3,Slogp4,Slogp5,Slogp6,Slogp7,Smr3,Smr4,Smr5,Smr6])
            




            
            

    

In [10]:
train_data1=pd.DataFrame(descriptors, columns=['array_id','fp','expected','mw','logp','TPSA','hbd','hba','n_rot_bonds','n_aromatic_rings','n_rings','n_heavy_atoms','frac_sp3','Slog3','Slog4','Slog5','Slog6','Slog7','Smr3','Smr4','Smr5','Smr6'])
test_data1=pd.DataFrame(descriptors1, columns=['array_id','fp','mw','logp','TPSA','hbd','hba','n_rot_bonds','n_aromatic_rings','n_rings','n_heavy_atoms','frac_sp3','Slog3','Slog4','Slog5','Slog6','Slog7','Smr3','Smr4','Smr5','Smr6'])

In [11]:
train_data1.head()

,array_id,fp,expected,mw,logp,TPSA,hbd,hba,n_rot_bonds,n_aromatic_rings,...,frac_sp3,Slog3,Slog4,Slog5,Slog6,Slog7,Smr3,Smr4,Smr5,Smr6
0,1644,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,317.599,4.5999,40.46,2,2,2,2,...,0.142857,0.000000,0.000000,17.044809,48.530937,0.00000,0.00000,0.000000,9.710442,0.000000
1,2451,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",2,156.269,3.3260,17.07,0,1,7,0,...,0.900000,4.794537,0.000000,58.793226,0.000000,0.00000,0.00000,0.000000,58.793226,0.000000
2,1384,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,362.086,4.3482,0.00,0,0,18,0,...,1.000000,0.000000,0.000000,116.580620,0.000000,0.00000,0.00000,0.000000,116.580620,27.184857
3,16,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,255.665,0.6879,83.66,1,5,3,1,...,0.333333,6.544756,10.114318,5.563451,23.321982,5.15311,15.30968,4.992405,6.544756,13.089513
4,1856,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,149.894,-5.9920,0.00,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000


In [12]:
test_data1.head()

,array_id,fp,mw,logp,TPSA,hbd,hba,n_rot_bonds,n_aromatic_rings,n_rings,...,frac_sp3,Slog3,Slog4,Slog5,Slog6,Slog7,Smr3,Smr4,Smr5,Smr6
0,1682,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",164.248,2.99812,20.23,1,1,0,1,1,...,0.454545,5.414990,6.923737,31.898115,18.199101,0.000000,0.000000,0.000000,33.109939,0.000000
1,1656,"[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",431.452,0.19280,166.54,2,10,7,2,2,...,0.285714,19.860544,0.000000,6.923737,34.317388,0.000000,19.674030,0.000000,16.845181,25.289237
2,36,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",696.264,6.61662,87.54,1,6,8,4,7,...,0.444444,15.438282,18.558179,72.733437,65.559154,5.022633,24.072992,0.000000,80.305485,26.681941
3,1850,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",201.244,-2.93050,74.54,0,4,0,0,1,...,0.250000,19.281787,0.000000,6.923737,11.835185,0.000000,0.000000,0.000000,6.923737,0.000000
4,30,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",418.574,4.58560,72.83,1,5,6,0,3,...,0.760000,19.062800,29.086615,73.143616,23.801165,0.000000,0.000000,29.086615,91.455515,0.000000


In [13]:
fp_list=[]
for i in range(train_data1.shape[0]):
    array=np.array(train_data1['fp'][i])
    fp_i=pd.DataFrame(array)
    fp_i=fp_i.T
    fp_list.append(fp_i)

len(fp_list)

75377

In [14]:
fp_list1=[]
for i in range(test_data1.shape[0]):
    array=np.array(test_data1['fp'][i])
    fp1_i=pd.DataFrame(array)
    fp1_i=fp1_i.T
    fp_list1.append(fp1_i)

len(fp_list1)

10994

In [15]:
fp_list[0]

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
df_fp_list=pd.concat(fp_list,ignore_index=True)
df_fp_list

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75372,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
75373,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
75374,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
75375,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [17]:
df_fp_list1=pd.concat(fp_list1,ignore_index=True)
df_fp_list1

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10989,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
10990,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10991,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
10992,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
train_final=df_fp_list.join(train_data1,how="outer")
train_final.head()

,0,1,2,3,4,5,6,7,8,9,...,frac_sp3,Slog3,Slog4,Slog5,Slog6,Slog7,Smr3,Smr4,Smr5,Smr6
0,0,1,0,0,0,0,0,0,0,0,...,0.142857,0.000000,0.000000,17.044809,48.530937,0.00000,0.00000,0.000000,9.710442,0.000000
1,0,0,0,0,0,0,0,0,0,0,...,0.900000,4.794537,0.000000,58.793226,0.000000,0.00000,0.00000,0.000000,58.793226,0.000000
2,0,0,0,0,0,0,0,0,0,0,...,1.000000,0.000000,0.000000,116.580620,0.000000,0.00000,0.00000,0.000000,116.580620,27.184857
3,0,0,0,1,0,0,0,0,0,0,...,0.333333,6.544756,10.114318,5.563451,23.321982,5.15311,15.30968,4.992405,6.544756,13.089513
4,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000


In [19]:
test_final=df_fp_list1.join(test_data1,how="outer")
test_final.head()

,0,1,2,3,4,5,6,7,8,9,...,frac_sp3,Slog3,Slog4,Slog5,Slog6,Slog7,Smr3,Smr4,Smr5,Smr6
0,0,0,0,0,0,0,0,0,0,0,...,0.454545,5.414990,6.923737,31.898115,18.199101,0.000000,0.000000,0.000000,33.109939,0.000000
1,1,0,0,0,0,0,0,1,0,0,...,0.285714,19.860544,0.000000,6.923737,34.317388,0.000000,19.674030,0.000000,16.845181,25.289237
2,0,0,0,0,0,0,0,0,0,0,...,0.444444,15.438282,18.558179,72.733437,65.559154,5.022633,24.072992,0.000000,80.305485,26.681941
3,0,0,0,0,0,0,0,0,0,0,...,0.250000,19.281787,0.000000,6.923737,11.835185,0.000000,0.000000,0.000000,6.923737,0.000000
4,1,0,0,0,0,1,0,0,0,0,...,0.760000,19.062800,29.086615,73.143616,23.801165,0.000000,0.000000,29.086615,91.455515,0.000000


In [20]:
train_final=train_final.drop('fp',axis=1)
test_final=test_final.drop('fp',axis=1)


In [28]:
X_train = train_final.drop("expected", axis = 1)
Y_train = train_final["expected"]
X_test  = test_final

X_train['array_id'] = X_train['array_id'].astype(int)
X_test['array_id'] = X_test['array_id'].astype(int)

X_train.shape, Y_train.shape, X_test.shape

((75377, 532), (75377,), (10994, 532))

In [ ]:
clf = AutoML()
clf.fit(X_train, y_train, task="classification", time_budget=3600, estimator_list=['lgbm'])

In [30]:
y_pred_train = clf.predict(X_train)
y_pred = clf.predict(X_test)

In [31]:
print('Accuracy: %.3f' % metrics.accuracy_score(Y_train, y_pred_train))

Accuracy: 1.000


In [32]:
print(metrics.classification_report(Y_train, y_pred_train))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00     10658
           2       1.00      1.00      1.00     64719

    accuracy                           1.00     75377
   macro avg       1.00      1.00      1.00     75377
weighted avg       1.00      1.00      1.00     75377



In [33]:
output = pd.DataFrame({'Id': test_data2['Id'], 'Predicted': y_pred})
output.to_csv('submission2_a_all.csv',index=False)